#Initial Downloads and imports

In [0]:
 !pip install pydrive --upgrade
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth) 

Requirement already up-to-date: pydrive in /usr/local/lib/python3.6/dist-packages (1.3.1)


In [0]:
# !pip install yfinance --upgrade --no-cache-dir
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

# import yfinance as yf 
# print(yf.__version__)

from datetime import datetime
from dateutil.parser import parse

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# import gensim
# import warnings
# warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
# warnings.simplefilter(action='ignore', category=FutureWarning)
# from gensim.models.keyedvectors import KeyedVectors
# from gensim.models import Word2Vec
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# from gensim.models.phrases import Phrases, Phraser

# import nltk
# nltk.download('punkt')
# from nltk import word_tokenize

1.14.0


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

path = '/content/drive/My Drive/Colab Notebooks/Text_Data.csv'
path2 = '/content/drive/My Drive/Colab Notebooks/Emotion_Data.csv'
path3 = '/content/drive/My Drive/Colab Notebooks/title_content.csv'
final_text = pd.read_csv(path, encoding='ISO-8859-1', engine='python', error_bad_lines=False)
df = pd.read_csv(path2, encoding='ISO-8859-1', engine='python', error_bad_lines=False)
ungrouped_text = pd.read_csv(path3, encoding='ISO-8859-1', engine='python', error_bad_lines=False)

final_text.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
ungrouped_text.drop('Unnamed: 0', axis=1, inplace=True)

df['Date'] = final_text['Date']
final_text = final_text.merge(df, how='inner', on='Date')

#ELMo

In [0]:
!pip install tensorflow
!pip install tensorflow-hub
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
print('done')

done


In [0]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

The output is a 3 dimensional tensor of shape (1, 8, 1024):

The first dimension of this tensor represents the number of training samples. This is 1 in our case
The second dimension represents the maximum length of the longest string in the input list of strings. Since we have only 1 string in our input list, the size of the 2nd dimension is equal to the length of the string – 8
The third dimension is equal to the length of the ELMo vector
Hence, every word in the input sentence has an ELMo vector of size 1024.

In [0]:
def elmo_vectors_doc(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    return sess.run(tf.reduce_mean(embeddings,1))

In [0]:
def elmo_vectors_word(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(embeddings)

In [0]:
batches = [ungrouped_text[i:i+100] for i in range(0,ungrouped_text.shape[0],100)]

In [0]:
ungrouped_text.shape

(88689, 3)

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# batch1 = batches[0:50]
# batch2 = batches[50:150]
# batch3 = batches[150:250]
# batch4 = batches[250:350]
# batch5 = batches[350:450]
# batch6 = batches[450:550]
# batch7 = batches[550:600]
# batch8 = batches[600:650]
# batch9 = batches[650:700]
# batch10 = batches[700:750]
# batch11 = batches[750:800]
# batch12 = batches[800:850]
batch13 = batches[850:]



In [0]:
import time
start = time.time()

# Extract ELMo embeddings
elmo_titles_batch13 = [elmo_vectors_doc(x['title']) for x in batch13]
# elmo_content = [elmo_vectors(x['content']) for x in list_content]
# elmo_titles = [elmo_vectors(x['title']) for x in list_titlesTEST]

done = time.time()
elapsed = done - start
print(elapsed)

219.73166823387146


In [0]:
elmo_titles_batch1

In [0]:
np.save("drive/My Drive/Colab Notebooks/elmo_titles_batch13", elmo_titles_batch13)

In [0]:
elmo_titles_new = np.concatenate(elmo_titles, axis = 0)
elmo_content_new = np.concatenate(elmo_content, axis = 0)

print('done')

In [0]:
# save elmo_titles_new
pickle_out = open("elmo_titles.pickle","wb")
pickle.dump(elmo_titles_new, pickle_out)
file = drive.CreateFile({'title': 'elmo_titles.pickle'})
file.SetContentFile('elmo_titles.pickle')
file.Upload()
pickle_out.close()

# save elmo_content_new
pickle_out = open("elmo_content.pickle","wb")
pickle.dump(elmo_content_new, pickle_out)
file = drive.CreateFile({'title': 'elmo_content.pickle'})
file.SetContentFile('elmo_content.pickle')
file.Upload()
pickle_out.close()

In [0]:
# load elmo_titles_new
pickle_in = open("elmo_titles.pickle", "rb")
elmo_titles_new = pickle.load(pickle_in)

# load elmo_content
pickle_in = open("elmo_content.pickle", "rb")
elmo_content_new = pickle.load(pickle_in)

In [0]:
elmo_titles_new.shape

#Vectorisation

In [0]:
def get_mean_vector(model, words):
  tokens = nltk.word_tokenize(words)
  accepted_tokens = [word for word in tokens if word in model.wv.vocab] #only accept words that are in word2vec vocabular
  vectors = [model[word] for word in accepted_tokens]# list word embeddings for each word in a given document
  if not vectors:
    mean_vector = np.zeros(300) # create a zero vector if the list of embeddings is empty
  else:
    mean_vector = np.mean(vectors, axis = 0) # average all embeddings in the list as a single vector
  return mean_vector

  
def vectorize(doc_lst, model):
  vectors = []
  for string in doc_lst:
    vectors.append(get_mean_vector(model, string))
  return vectors

In [0]:
title_vectors = vectorize(title_strings, w2v_model)
content_vectors = vectorize(content_strings, w2v_model)

In [0]:
ungrouped_text['title_vectors'] = pd.Series(title_vectors)
ungrouped_text['content_vectors'] = pd.Series(content_vectors)

In [0]:
result_df = ungrouped_text.groupby('date')['title_vectors'].apply(np.mean).to_frame('Title')
result_df2 = ungrouped_text.groupby('date')['content_vectors'].apply(np.mean).to_frame('Content')

In [0]:
result_df['Content'] = pd.Series(result_df2['Content'])
result_df = result_df.reset_index().rename(columns={"date" : "Date"})

In [0]:
result_df.to_csv('Vectorised_Text_Data.csv')
!cp Vectorised_Text_Data.csv drive/My\ Drive/Colab\ Notebooks/